## オブザーバビリティを高める機能

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [2]:
import os, warnings, time

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

### プロンプト

各トレースを同一グループにするために、セッションIDをUUID v4ベースで生成する

In [3]:
from langfuse import Langfuse

# Initialize Langfuse client
langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
    )


In [41]:
from llm_guard.input_scanners import Anonymize
from llm_guard.input_scanners.anonymize_helpers import BERT_LARGE_NER_CONF
from llm_guard.output_scanners import Deanonymize
from llm_guard.vault import Vault
from langfuse.decorators import observe
from langchain_core.runnables import RunnableLambda

vault = Vault()


def anonymize(input: dict) -> dict:
  scanner = Anonymize(vault, preamble="Insert before prompt", allowed_names=["John Doe"], hidden_names=["Test LLC"],
                    recognizer_conf=BERT_LARGE_NER_CONF, language="en")
  sanitized_prompt, is_valid, risk_score = scanner.scan(input['prompt'])
  return {"sanitized_prompt":sanitized_prompt}
runnable_add_one = RunnableLambda(anonymize)

#@observe()
#def deanonymize(sanitized_prompt: str, answer: str):
#  scanner = Deanonymize(vault)
#  sanitized_model_output, is_valid, risk_score = scanner.scan(sanitized_prompt, answer)
#
#  return sanitized_model_output

In [42]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langfuse.decorators import observe
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
)

#@observe()
def summarize_transcript(prompt: str):
  #sanitized_prompt = anonymize(prompt)

  prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","Summarize the given court transcript."),
        ("user","{sanitized_prompt}"),
    ]
  )
  model = ChatCohere(
      model="command-r-plus",
      temperature=1,
  )
  chain = runnable_add_one| prompt_template | model

  #sanitized_model_output = deanonymize(sanitized_prompt, answer)
  chain.invoke(
    {"prompt":prompt},
    config={"callbacks": [callback_handler]},
  )

  return

In [45]:
summarize_transcript("Jon Tierについて教えて欲しい。それかJohn Doeでもいいんだけど、Test LLCもそうだけど、彼はいま何をしてますか？")

validate_env
2024-10-23 04:40:49 [debug    ] No entity types provided, using default default_entities=['CREDIT_CARD', 'CRYPTO', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'US_SSN', 'US_BANK_NUMBER', 'CREDIT_CARD_RE', 'UUID', 'EMAIL_ADDRESS_RE', 'US_SSN_RE']


Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2024-10-23 04:40:49 [debug    ] Initialized NER model          device=device(type='cpu') model=Model(path='dslim/bert-large-NER', subfolder='', revision='13e784dccceca07aee7a7aab4ad487c605975423', onnx_path='dslim/bert-large-NER', onnx_revision='13e784dccceca07aee7a7aab4ad487c605975423', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'aggregation_strategy': 'simple', 'ignore_labels': ['O', 'CARDINAL']}, tokenizer_kwargs={'model_input_names': ['input_ids', 'attention_mask']})
2024-10-23 04:40:49 [debug    ] Loaded regex pattern           group_name=CREDIT_CARD_RE
2024-10-23 04:40:49 [debug    ] Loaded regex pattern           group_name=UUID
2024-10-23 04:40:49 [debug    ] Loaded regex pattern           group_name=EMAIL_ADDRESS_RE
2024-10-23 04:40:49 [debug    ] Loaded regex pattern           group_name=US_SSN_RE
2024-10-23 04:40:49 [debug    ] Loaded regex pattern           group_name=BTC_ADDRESS
2024-10-23 0

プロンプトテンプレートの作成